In [ ]:
import functools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import config
import pytorch_lightning as pl
import torchmetrics
import torchvision.models as models
from math import ceil

torch.manual_seed(42)

ray.init(num_cpus=config.NUM_CPUS, num_gpus=config.NUM_GPUS, log_to_driver=False)
# %config InlineBackend.figure_formats = ['svg'] # Make plots high resolution
plt.rcParams['figure.figsize'] = (12, 12) # Increase figure size